In [7]:
import summa_plot as sp
import pysumma.Simulation
import importlib
import time
import pandas as pd
importlib.reload(pysumma.Simulation)

sites = ['Amplero', 'Blodgett', 'Bugac', 'ElSaler', 'ElSaler2', 'Espirra', 'FortPeck', 
         'Harvard', 'Hesse', 'Howard', 'Howlandm', 'Hyytiala', 'Kruger', 'Loobos', 'Merbleue',
         'Mopane', 'Palang', 'Sylvania', 'Tumba', 'UniMich']
fman_template = "/pool0/data/tushark/PLUMBER_data/sites/{}/settings/summa_zFileManager_{}.txt"
fman_dict = {s: fman_template.format(s, s) for s in sites}

def run_summa(site):
    s = pysumma.Simulation.Simulation(fman_dict[site])
    #s.decision_obj.stomResist.value = 'simpleResistance'
    # s.decision_obj.groundwatr = 'noXplict'
    #s.decision_obj.bcLowrSoiH.value = 'zeroFlux'
    #s.decision_obj.__dict__['rootProfil'].value = 'powerLaw'
    #s.decision_obj.__dict__['hc_profile'].value = 'pow_prof'
    set_decision(s, 'groundwatr', 'qTopmodl')
    set_decision(s, 'bcLowrSoiH', 'zeroFlux')
    set_decision(s, 'f_Richards', 'mixdform')
    set_decision(s, 'rootProfil', 'powerLaw')
    set_decision(s, 'hc_profile', 'pow_prof')
    s.executable = '/opt/local/bin/summa.exe'
    s.library_path = '/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH'
    additional_args = ['export OMP_NUM_THREADS=8']
    return s.execute('test_simpleResistance_2', 'local', additional_args)

def monitor(proc):
    while proc.poll() is not None:
        time.sleep(1)
    return proc.stdout.read(), proc.stderr.read()

def set_decision(simulation, decision_key, decision_value):
    simulation.decision_obj.__dict__[decision_key].value = decision_value
    

In [8]:
proc, out_file = run_summa('Bugac')
out, err = monitor(proc)

export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8 && /opt/local/bin/summa.exe -p never -s test_simpleResistance_2 -m /pool0/data/tushark/PLUMBER_data/sites/Bugac/settings/summa_zFileManager_Bugac.txt


In [9]:
out

b"file_suffix is 'test_simpleResistance_2'.\nfile_master is '/pool0/data/tushark/PLUMBER_data/sites/Bugac/settings/summa_zFileManager_Bugac.txt'.\nstart at 14:54:14\nName of Model Output control file: /data/tushark/PLUMBER_data/sites/Bugac/settings/meta/summa_zModelOutput.txt\ndecisions file =  /data/tushark/PLUMBER_data/sites/Bugac/settings/summa_zDecisions_Bugac.txt\n   1 simulStart: 2003-01-01 00:30\n   2 simulFinsh: 2007-01-01 00:00\n   3 soilCatTbl: ROSETTA\n   4 vegeParTbl: USGS\n   5 soilStress: NoahType\n   6 stomResist: simpleResistance\n   7 bbTempFunc: q10Func\n   8 bbHumdFunc: humidLeafSurface\n   9 bbElecFunc: linear\n  10 bbCO2point: origBWB\n  11 bbNumerics: NoahMPsolution\n  12 bbAssimFnc: minFunc\n  13 bbCanIntg8: constantScaling\n  14 num_method: itertive\n  15 fDerivMeth: analytic\n  16 LAI_method: monTable\n  17 cIntercept: storageFunc\n  18 f_Richards: mixdform\n  19 groundwatr: qTopmodl\n  20 hc_profile: pow_prof\n  21 bcUpprTdyn: nrg_flux\n  22 bcLowrTdyn: presTe

In [4]:
err

b'Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG\nSTOP 1\n'

In [5]:
def run_summa2(site):
    s = pysumma.Simulation.Simulation(fman_dict[site])
    s.decision_obj.stomResist.value = 'simpleResistance'
    # s.decision_obj.groundwatr.value = 'qTopmodl'
    s.decision_obj.hc_profile.value = 'constant'
    s.executable = '/opt/local/bin/summa.exe'
    s.library_path = '/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH'
    additional_args = ['export OMP_NUM_THREADS=8']
    return s.execute('test_simpleResistance_3', 'local', additional_args)

In [ ]:
proc2, out_file2 = run_summa2('Bugac')
out2, err2 = monitor(proc)

export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8 && /opt/local/bin/summa.exe -p never -s test_simpleResistance_3 -m /pool0/data/tushark/PLUMBER_data/sites/Bugac/settings/summa_zFileManager_Bugac.txt


In [2]:
def run_combinations(dataframe, sites):
    '''
    This function takes a dataframe with summa
    decision points as column names, and with each
    row representing a summa run with the appropriate
    decision options
    It also takes an array_like object sites which just
    lists the names of the sites for which each summa
    combination should be run at.
    '''
    result = []
    name_template = "run1_index{}"
    
    # assuming we want to run all sites
    # another option would be to have a column in the 
    # dataframe representing the site for each run
    for site in sites:
        
        # set up a simulation object for the site
        s = pysumma.Simulation.Simulation(fman_dict[site])
        s.executable = '/opt/local/bin/summa.exe'
        s.library_path = '/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH'
        additional_args = ['export OMP_NUM_THREADS=8']
        
        # for each option combination, set the appropriate
        # values in the decision object, then run
        for index in dataframe.index.values:
            for decision in dataframe.columns.values:
                
                print(index, decision, dataframe[decision][index])
                s.decision_obj.__dict__[decision].value = dataframe[decision][index]
                
            # need naming convention for output files
            #name = name_template.format(index)
            proc, out_file = s.execute('test_run_4', 'local', additional_args)
            out, err = monitor(proc)
            result.append(out_file)
    
    return result
                

In [16]:
def run_combinations2(dataframe, sites):
    '''
    This function takes a dataframe with summa
    decision points as column names, and with each
    row representing a summa run with the appropriate
    decision options
    It also takes an array_like object sites which just
    lists the names of the sites for which each summa
    combination should be run at.
    '''
    result = []
    name_template = "run1_index{}"
    
    # assuming we want to run all sites
    # another option would be to have a column in the 
    # dataframe representing the site for each run
    for site in sites:
        
        # for each option combination, set the appropriate
        # values in the decision object, then run
        for index in dataframe.index.values:
            
            # set up a simulation object for the site
            s = pysumma.Simulation.Simulation(fman_dict[site])
            s.executable = '/opt/local/bin/summa.exe'
            s.library_path = '/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH'
            additional_args = ['export OMP_NUM_THREADS=8']
            
            for decision in dataframe.columns.values:
                
                print(index, decision, dataframe[decision][index])
                s.decision_obj.__dict__[decision].value = dataframe[decision][index]
                
            # need naming convention for output files
            #name = name_template.format(index)
            proc, out_file = s.execute('test_run_5', 'local', additional_args)
            out, err = monitor(proc)
            result.append(out_file)
    
    return result
                

In [3]:
df = pd.DataFrame(columns=['stomResist'])
df1 = pd.DataFrame(columns=['stomResist', 'groundwatr', 'hc_profile'])

In [4]:
dict2 = {'stomResist':'Jarvis'}
df3 = df.append(dict2, ignore_index=True)
df3

,stomResist
0,Jarvis


In [5]:
dict1 = {'stomResist':'Jarvis', 'groundwatr':'qTopmodl', 'hc_profile':'constant'}
dicts = [{'stomResist':'Jarvis', 'groundwatr':'qTopmodl', 'hc_profile':'constant'},
         {'stomResist':'BallBerry', 'groundwatr':'qTopmodl', 'hc_profile':'pow_prof'},
         {'stomResist':'simpleResistance', 'groundwatr':'qTopmodl', 'hc_profile':'constant'}]
df2 = df1.append(dict1, ignore_index=True)
df2

,stomResist,groundwatr,hc_profile
0,Jarvis,qTopmodl,constant


In [6]:
df4 = df1.append(dicts, ignore_index=True)
df4

,stomResist,groundwatr,hc_profile
0,Jarvis,qTopmodl,constant
1,BallBerry,qTopmodl,pow_prof
2,simpleResistance,qTopmodl,constant


In [7]:
sites_short = ['Amplero', 'Blodgett']

In [7]:
output = run_combinations(df3, sites_short)

0 stomResist Jarvis
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8 && /opt/local/bin/summa.exe -p never -s run1_index0 -m /pool0/data/tushark/PLUMBER_data/sites/Amplero/settings/summa_zFileManager_Amplero.txt
0 stomResist Jarvis
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8 && /opt/local/bin/summa.exe -p never -s run1_index0 -m /pool0/data/tushark/PLUMBER_data/sites/Blodgett/settings/summa_zFileManager_Blodgett.txt


In [8]:
output2 = run_combinations(df2, sites_short)

0 stomResist Jarvis
0 groundwatr qTopmodl
0 hc_profile constant
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8 && /opt/local/bin/summa.exe -p never -s test_run_4 -m /pool0/data/tushark/PLUMBER_data/sites/Amplero/settings/summa_zFileManager_Amplero.txt
0 stomResist Jarvis
0 groundwatr qTopmodl
0 hc_profile constant
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8 && /opt/local/bin/summa.exe -p never -s test_run_4 -m /pool0/data/tushark/PLUMBER_data/sites/Blodgett/settings/summa_zFileManager_Blodgett.txt


In [9]:
output2

['/data/tushark/PLUMBER_data/sites/Amplero/summa_output/Amplero_output_test_run_4_timestep.nc',
 '/data/tushark/PLUMBER_data/sites/Blodgett/summa_output/Blodgett_output_test_run_4_timestep.nc']

In [10]:
output3 = run_combinations(df2, ['Espirra', 'FortPeck'])

0 stomResist Jarvis
0 groundwatr qTopmodl
0 hc_profile constant
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8 && /opt/local/bin/summa.exe -p never -s test_run_4 -m /pool0/data/tushark/PLUMBER_data/sites/Espirra/settings/summa_zFileManager_Espirra.txt
0 stomResist Jarvis
0 groundwatr qTopmodl
0 hc_profile constant
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8 && /opt/local/bin/summa.exe -p never -s test_run_4 -m /pool0/data/tushark/PLUMBER_data/sites/FortPeck/settings/summa_zFileManager_FortPeck.txt


In [12]:
output4 = run_combinations(df3, sites_short)

0 stomResist Jarvis
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8 && /opt/local/bin/summa.exe -p never -s test_run_4 -m /pool0/data/tushark/PLUMBER_data/sites/Amplero/settings/summa_zFileManager_Amplero.txt
0 stomResist Jarvis
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8 && /opt/local/bin/summa.exe -p never -s test_run_4 -m /pool0/data/tushark/PLUMBER_data/sites/Blodgett/settings/summa_zFileManager_Blodgett.txt


In [17]:
output5 = run_combinations2(df3, ['Espirra', 'FortPeck'])

0 stomResist Jarvis
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8 && /opt/local/bin/summa.exe -p never -s test_run_5 -m /pool0/data/tushark/PLUMBER_data/sites/Espirra/settings/summa_zFileManager_Espirra.txt
0 stomResist Jarvis
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8 && /opt/local/bin/summa.exe -p never -s test_run_5 -m /pool0/data/tushark/PLUMBER_data/sites/FortPeck/settings/summa_zFileManager_FortPeck.txt


In [18]:
a,b = run_summa('FortPeck')

export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8
export LD_LIBRARY_PATH="/opt/local/lib:/opt/local/lib64:$LD_LIBRARY_PATH" && export OMP_NUM_THREADS=8 && /opt/local/bin/summa.exe -p never -s test_simpleResistance_new -m /pool0/data/tushark/PLUMBER_data/sites/FortPeck/settings/summa_zFileManager_FortPeck.txt


In [ ]:
out, err = monitor(a)

In [ ]:
for site in sites:
    run_summa(site)

In [ ]:
for site in sites:
    run_summa2(site)

In [ ]:
err